# GDMA Project
Author: Julian Schelb (1069967)

In [1]:
from neo4j import GraphDatabase
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Connection to the database instance

In [2]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "subatomic-shrank-Respond"))
database_name = "cddb"
session = driver.session(database = database_name)

### Task 4: Searching and Ranking

Implement a simple search engine that enables search by artist, album and
song name/title. The results must be ranked based on importance. It is up to
you to come up with how the importance of each result is computed and you
must justify your decision (it goes without saying that you need to come up
with a meaningful definition). However, the importance should ideally take into
account user preferences/likes. As such, this task is split in two parts:

**1. Write a Cypher query that adds a relationship :LIKES between a node with**

label :User and an artist, album, or song. Every user should be identified
just by a numerical userID (no more information is necessary). If a user
already exists in the system, no additional node should be added. After
coming up with the necessary Cypher query, add a significant number of
users and likes.

**2. Implement a simple Python function that has the following arguments:**

- the userID of the user submitting the search (the user ID may not
exist in the database),
- a string that contains one or more keywords for the search, and
- an optional argument that indicates whether the search is on all or
a specific field, i.e., artist, album, song.
The search must return exactly 10 results.

Python must only be used to call the database. You should not write any
code in Python that implements functionality necessary for the task. However,
submitting multiple queries in the same function call is allowed. Also, for this
task of the project, you are not only allowed but also encouraged to use functions
from the GDS library of Neo4j. Hence, before making any decisions, have a
careful look at the available functions. Again, you have to justify the use of any
function that you employ

In [3]:
query = """
MATCH (c:CD)-[r:CONTAINS]->(ar:Artist)
WHERE c.ayear = 2000
RETURN DISTINCT ar.artist
"""
        
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data

,ar.artist
0,the frequency benders
1,boyz ii men
2,adriana calcanhoto
3,syl johnson
4,cheo feliciano
...,...
5619,garry harrison
5620,catuaba com amendoim
5621,jimmy powells
5622,ofra haza


#### Processing

**Ideas:** 

Inplicit Meassures (to avoid cold start problem):
- Centrality in subgraph of artists, albums, songs according to search input (Use exact match and partial match) (and maybe liked nodes?)
    1) Select Subgraph by filtering artbist, albums and songs by input
    2) Calculate centrality (Multiple meassures)
    3) Cd with higher centrality is considered better
    

User Preference:

- Use Nod2Vec Embedding to compute Similarity to already liked Nodes:
    1) Include similar nodes to already liked ones
    2) Average similarity of artists/albums/sing nodes ... compared to previous likes
- Use average Distance to liked Nodes. (basically already included)
- centrality in the subgraph ob liked nodes (basically already included)

Output:

cd id, contained songs, contained artists, contained albums, centralities, cumulated scores

#### Adding Example Users

**Function to create Example User:**

In [53]:
def createExampleUser(user_id = 1, genre = "rock", limit = 50):
    
    ####### DELETE USER NODE #######
    query1 = """
    MATCH (u:User)
    WHERE u.id = $user_id
    DETACH DELETE u
    """
    
    ####### CREATE USER NODE #######
    query2 = """
    MERGE (u:User {id:  $user_id})
    RETURN u.id as user_id
    """
    
    ####### LINK TO LIKED SONGS #######
    query3 = """
    MATCH (g:Genre)<-[r:BELONGS_TO]-(c:CD)
    MATCH (c)-[r2:CONTAINS]->(s:Song)
    WHERE g.genre = $genre
    WITH s 
    LIMIT $limit
    MATCH (u:User)
    WHERE u.id = $user_id
    MERGE (u)-[r3:LIKES]->(s)
    """
    
    ####### LINK TO LIKED ARTISTS #######
    query4 = """
    MATCH (g:Genre)<-[r:BELONGS_TO]-(c:CD)
    MATCH (c)-[r2:CONTAINS]->(t:Artist)
    WHERE g.genre = $genre
    WITH t 
    LIMIT $limit
    MATCH (u:User)
    WHERE u.id = $user_id
    MERGE (u)-[r3:LIKES]->(t)
    """
    
    ####### LINK TO LIKED ALBUMS #######
    query5 = """
    MATCH (g:Genre)<-[r:BELONGS_TO]-(c:CD)
    MATCH (c)-[r2:CONTAINS]->(t:Album)
    WHERE g.genre = $genre
    WITH t 
    LIMIT $limit
    MATCH (u:User)
    WHERE u.id = $user_id
    MERGE (u)-[r3:LIKES]->(t)
    """

    with driver.session(database = database_name) as session:
        session.run(query1, user_id = user_id)
        session.run(query2, user_id = user_id)
        session.run(query3, user_id = user_id, genre = genre, limit = limit)
        session.run(query4, user_id = user_id, genre = genre, limit = limit)
        session.run(query5, user_id = user_id, genre = genre, limit = limit)
    

**Adding some Likes to model User Preference:**

In [54]:
# User 1 likes "Rock" music
createExampleUser(user_id = 1, genre = "rock", limit = 50)
# User 2 likes "classic" music
# createExampleUser(user_id = 2, genre = "classic", limit = 5)
# User 3 likes "pop" music
# createExampleUser(user_id = 3, genre = "pop", limit = 50)
# User 4 likes "hip-hop" music
# createExampleUser(user_id = 4, genre = "hip-hop", limit = 50)
# User 5 likes "hip-hop" music
# createExampleUser(user_id = 5, genre = "hard rock", limit = 50)

In [9]:
# User 2 likes "classic" music
# createExampleUser(user_id = 0, genre = "classic", limit = 500)

***

#### Implementing Search Engine

##### Feature 1: User Preference

In [55]:
query = """
// DELETE EXISTING PROJECTION
CALL gds.graph.drop('searchdomain_preference', false) 
YIELD graphName 
RETURN graphName
"""

dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data

,graphName
0,searchdomain_preference


In [56]:
query = """
// CREATE NEW PROJECTION WITH SEARCH RELEVANT SUB GRAPH
CALL gds.graph.project.cypher(
  'searchdomain_preference',
  ' // Liked Artists, Albums and Songs
    MATCH (u:User)-[:LIKES]->(n) 
    WHERE u.id = 1
        AND (n:Song OR n:Album OR n:Artist) 
    RETURN id(n) AS id, labels(n) AS labels 
    LIMIT 100000
    
    UNION
    
    // CDs linked to liked Artists, Albums and Songs
    MATCH (u:User)-[:LIKES]->(x)-[:APPEARED_ON]->(n:CD) 
    WHERE u.id = 1 
    RETURN id(n) AS id, labels(n) AS labels 
    LIMIT 100000',
    
    'MATCH (u:User)-[:LIKES]->(n)
    WHERE u.id = 1
    AND (n:CD OR n:Song OR n:Album OR n:Artist) 
    MATCH (n)-[r:APPEARED_ON]->(m:CD) 
    RETURN id(n) AS source, id(m) AS target, type(r) AS type 
    LIMIT 100000' 
)
YIELD
  graphName, nodeCount AS nodes, relationshipCount AS rels
RETURN graphName, nodes, rels
"""

dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data

,graphName,nodes,rels
0,searchdomain_preference,2071,2108


In [57]:
# https://neo4j.com/docs/graph-data-science/current/algorithms/eigenvector-centrality/
query = """
CALL gds.eigenvector.mutate('searchdomain_preference',  {
  mutateProperty: 'score_eig'
})
YIELD centralityDistribution, nodePropertiesWritten, ranIterations
RETURN centralityDistribution.min AS minimumScore, centralityDistribution.mean AS meanScore, nodePropertiesWritten
"""
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(30)

,minimumScore,meanScore,nodePropertiesWritten
0,0.000761,0.018063,2071


In [58]:
# https://neo4j.com/docs/graph-data-science/current/algorithms/closeness-centrality/#algorithms-closeness-centrality-examples-mutate
query = """
CALL gds.beta.closeness.mutate('searchdomain_preference',  {
  mutateProperty: 'score_cln'
})
YIELD centralityDistribution, nodePropertiesWritten
RETURN centralityDistribution.min AS minimumScore, centralityDistribution.mean AS meanScore, nodePropertiesWritten
"""
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(30)

,minimumScore,meanScore,nodePropertiesWritten
0,0.0,0.928054,2071


In [59]:
# https://neo4j.com/docs/graph-data-science/current/algorithms/degree-centrality/
query = """
CALL gds.degree.mutate('searchdomain_preference',  {
  mutateProperty: 'score_deg'
})
YIELD centralityDistribution, nodePropertiesWritten
RETURN centralityDistribution.min AS minimumScore, centralityDistribution.mean AS meanScore, nodePropertiesWritten
"""
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(30)

,minimumScore,meanScore,nodePropertiesWritten
0,0.0,1.017868,2071


In [60]:
query = """
CALL gds.graph.streamNodeProperties('searchdomain_preference', ['score_eig', 'score_deg', 'score_cln'])
YIELD nodeId, nodeProperty, propertyValue
WITH nodeId, gds.util.asNode(nodeId).id AS id, nodeProperty, propertyValue

ORDER BY nodeId, nodeProperty
WITH  nodeId, gds.util.asNode(nodeId).id as id, 
collect(nodeProperty) as properties , collect(propertyValue) as values, sum(propertyValue) as sum_values

MATCH (n:CD)
WHERE n.id = id
MATCH (n)-[:CONTAINS]->(ar:Artist)
MATCH (n)-[:CONTAINS]->(ab:Album)
MATCH (n)-[:CONTAINS]->(so:Song)
RETURN id, 
values, sum_values, 
collect(DISTINCT ar.artist) as artists,
collect(DISTINCT ab.album) as albums, 
collect(DISTINCT so.song) as songs
ORDER BY sum_values DESC
LIMIT 100
"""

dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(30)

,id,values,sum_values,artists,albums,songs
0,7490,"[0.0, 444.0, 0.0007609639559427433]",444.000761,[billie holiday],[the ultimate collection],"[the blues are brewin, i'll get by (as long as..."
1,1538,"[0.0, 421.0, 0.0007609639559427433]",421.000761,[mauv],[omniprésence],"[omniprésence [1], omniprésence [ ]]"
2,76,"[0.0, 256.0, 0.0007609639559427433]",256.000761,[eric clapton],[the great blues works],"[good morning little schoolgirl, five long yea..."
3,4721,"[0.0, 106.0, 0.0007609639559427433]",106.000761,[lifehouse],[hangingmoment],"[fairy tales and sandcastles, fool, what's wro..."
4,529,"[0.0, 74.0, 0.0007609639559427433]",74.000761,[jimi hendrix],[bbc sessions],"[driving south, hear my train a comin, foxy la..."
5,6312,"[0.0, 69.0, 0.0007609639559427433]",69.000761,[pink floyd],[animals],"[dogs, sheep, pigs on the wing [part 1], pigs ..."
6,4534,"[0.0, 48.0, 0.0007609639559427433]",48.000761,[depeche mode],[barrel of a gun],"[barrel of a gun (one inch punch mix);, barrel..."
7,717,"[0.0, 48.0, 0.0007609639559427433]",48.000761,[queen],[the platinum collection],"[breakthru, one vision, i want it all, i want ..."
8,902,"[0.0, 41.0, 0.0007609639559427433]",41.000761,[b'z],[banzai],[magnolia]
9,2652,"[0.0, 34.0, 0.0007609639559427433]",34.000761,[ben harper],[diamonds on the inside],"[strawberry fields forever, with my own two ha..."


In [31]:
query = """
CALL gds.graph.streamNodeProperty('searchdomain_preference', 'score_eig')
YIELD nodeId as nodeId_eig, propertyValue as score_eig

//CALL gds.graph.streamNodeProperty('searchdomain', 'score_deg')
//YIELD nodeId as nodeId_deg, propertyValue as propertyValue_deg

//CALL gds.graph.streamNodeProperty('searchdomain_preference', 'score_cln')
//YIELD nodeId as nodeId_cln, propertyValue as score_cln

WITH gds.util.asNode(nodeId_eig).id AS id, 
score_eig,  collect(score_eig)  as t//avg(score_eig) as score_eig_mean
RETURN *
ORDER BY score_eig DESC
"""
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(30)

,id,score_eig,t
0,53708,0.061017,[0.06101741687030329]
1,135693,0.061017,[0.06101741687030329]
2,109958,0.061017,[0.06101741687030329]
3,44549,0.061017,[0.06101741687030329]
4,47493,0.061017,[0.06101741687030329]
5,131914,0.061017,[0.06101741687030329]
6,116208,0.061017,[0.06101741687030329]
7,51143,0.061017,[0.06101741687030329]
8,53698,0.061017,[0.06101741687030329]
9,111637,0.061017,[0.06101741687030329]


In [61]:
query = """
CALL gds.eigenvector.stream('searchdomain_preference')
YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS nodeId, score
MATCH (n:CD)-[r:CONTAINS]->(a:Artist)
WHERE n.id = nodeId
SET n.eigenvector = score
RETURN n.id, score, 
collect(a.artist) as artists
ORDER BY score DESC
"""
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(30)

,n.id,score,artists
0,33062,0.255736,[ben harper]
1,167085,0.238738,[jannedaarc]
2,28698,0.221739,[ben harper]
3,166362,0.187743,[the coral]
4,11783,0.170744,[groundhogs]
5,13220,0.170744,[groundhogs]
6,16055,0.102751,[blankey jet city]
7,121360,0.102751,[natural punch drunker]
8,32936,0.085753,[ben harper]
9,10315,0.085753,[ben harper]


##### Feature 2: Content Match with Search Input

In [62]:
query = """
// DELETE EXISTING PROJECTION
CALL gds.graph.drop('searchdomain_content', false) 
YIELD graphName 
RETURN graphName
"""

dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data

,graphName
0,searchdomain_content


In [63]:
query = """
// CREATE NEW PROJECTION WITH SEARCH RELEVANT SUB GRAPH
CALL gds.graph.project.cypher(
  "searchdomain_content",
  
  " // Artists, Albums and Songs which match query
    MATCH (n) 
    WHERE 
    (n:Song OR n:Album OR n:Artist) 
    AND 
    (n.artist =~ \'.*jimi hendrix.*\' 
    OR n.album  =~ '.*Are You Experienced.*' 
    OR n.song  =~ '.*purple haze.*')
    RETURN id(n) AS id, labels(n) AS labels 
    LIMIT 1000
    
    UNION
    
    // CDS linked to Artists, Albums and Songs which match query
    MATCH (n)-[:APPEARED_ON]->(c:CD) 
    WHERE 
    (n:Song OR n:Album OR n:Artist) 
    AND 
    (n.artist =~ '.*jimi hendrix.*' 
    OR n.album  =~ '.*Are You Experienced.*' 
    OR n.song  =~ '.*purple haze.*')
    RETURN id(c) AS id, labels(c) AS labels 
    LIMIT 1000",
    
    "MATCH (n)-[r:APPEARED_ON]->(c:CD) 
    WHERE 
    (n:Song OR n:Album OR n:Artist) 
    AND 
    (n.artist =~ '.*jimi hendrix.*' 
    OR n.album  =~ '.*Are You Experienced.*' 
    OR n.song  =~ '.*purple haze.*')
    RETURN id(n) AS source, id(c) AS target, type(r) AS type 
    LIMIT 1000"
)
YIELD
  graphName, nodeCount AS nodes, relationshipCount AS rels
RETURN graphName, nodes, rels
"""


dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)]) 
dtf_data

,graphName,nodes,rels
0,searchdomain_content,140,162


In [64]:
# https://neo4j.com/docs/graph-data-science/current/algorithms/eigenvector-centrality/
query = """
CALL gds.eigenvector.mutate('searchdomain_content',  {
  mutateProperty: 'score_eig'
})
YIELD centralityDistribution, nodePropertiesWritten, ranIterations
RETURN centralityDistribution.min AS minimumScore, centralityDistribution.mean AS meanScore, nodePropertiesWritten
"""
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(30)

,minimumScore,meanScore,nodePropertiesWritten
0,0.002642,0.071719,140


In [43]:
# https://neo4j.com/docs/graph-data-science/current/algorithms/closeness-centrality/#algorithms-closeness-centrality-examples-mutate
query = """
CALL gds.beta.closeness.mutate('searchdomain_content',  {
  mutateProperty: 'score_cln'
})
YIELD centralityDistribution, nodePropertiesWritten
RETURN centralityDistribution.min AS minimumScore, centralityDistribution.mean AS meanScore, nodePropertiesWritten
"""
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(30)

,minimumScore,meanScore,nodePropertiesWritten
0,0.0,0.85,140


In [44]:
# https://neo4j.com/docs/graph-data-science/current/algorithms/degree-centrality/
query = """
CALL gds.degree.mutate('searchdomain_content',  {
  mutateProperty: 'score_deg'
})
YIELD centralityDistribution, nodePropertiesWritten
RETURN centralityDistribution.min AS minimumScore, centralityDistribution.mean AS meanScore, nodePropertiesWritten
"""
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(30)

,minimumScore,meanScore,nodePropertiesWritten
0,0.0,1.157146,140


In [45]:
query = """
CALL gds.graph.streamNodeProperties('searchdomain_content', ['score_eig', 'score_deg', 'score_cln'])
YIELD nodeId, nodeProperty, propertyValue
WITH nodeId, gds.util.asNode(nodeId).id AS id, nodeProperty, propertyValue

ORDER BY nodeId, nodeProperty
WITH  nodeId, gds.util.asNode(nodeId).id as id, 
collect(nodeProperty) as properties , collect(propertyValue) as values, sum(propertyValue) as sum_values

MATCH (n:CD)
WHERE n.id = id
MATCH (n)-[:CONTAINS]->(ar:Artist)
MATCH (n)-[:CONTAINS]->(ab:Album)
MATCH (n)-[:CONTAINS]->(so:Song)
RETURN id, 
values, sum_values, 
collect(DISTINCT ar.artist) as artists,
collect(DISTINCT ab.album) as albums, 
collect(DISTINCT so.song) as songs
ORDER BY sum_values DESC
LIMIT 100
"""

dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(45)

,id,values,sum_values,artists,albums,songs
0,529,"[0.0, 74.0, 0.002642306670565559]",74.002642,[jimi hendrix],[bbc sessions],"[driving south, hear my train a comin, foxy la..."
1,3248,"[0.0, 55.0, 0.002642306670565559]",55.002642,[tamba trio],[tamba trio],"[consolacao, batida diferente, imagem, canto d..."
2,452,"[0.0, 8.0, 0.002642306670565559]",8.002642,[the jimi hendrix experience],[bbc sessions],"[driving south (alternate take);, wait until t..."
3,20709,"[0.0, 5.0, 0.002642306670565559]",5.002642,[jimi hendrix experience],[the vpro archive - it must be dusty],"[mockingbird (mpeg1 video data);, spoken word,..."
4,2835,"[0.0, 2.0, 0.002642306670565559]",2.002642,[jimi hendrix (w. lonnie youngblood);],[two great experiencesremastered);],"[she's a fox (1);, groove maker 1 (3);, soul f..."
5,110321,"[0.0, 2.0, 0.002642306670565559]",2.002642,[rajyashree josyer shrikanth],[sangeetha sowrabha],"[brochevarevarura (khamas, adi, mysore vasudev..."
6,7923,"[1.0, 0.0, 0.3011216110364633]",1.301122,[signature licks],[jimi hendrix],"[fire (verse);, foxey lady (solo, slow);, fire..."
7,30138,"[1.0, 0.0, 0.18172988929010417]",1.181730,[jimi hendrix],[astro man box set],"[can you see me (pre unre);, purple haze (pre ..."
8,33321,"[1.0, 0.0, 0.18172988929010417]",1.181730,[jimi hendrix],[astro man(alchemy); - studio outtakes 1966-68],"[purple haze 1 (4);, 51st anniversary (5);, la..."
9,20709,"[1.0, 0.0, 0.18172988929010417]",1.181730,[jimi hendrix experience],[the vpro archive - it must be dusty],"[mockingbird (mpeg1 video data);, spoken word,..."


Combined Search Results:

In [72]:
query = """
CALL {

    CALL {
        CALL gds.eigenvector.stream('searchdomain_content')
        YIELD nodeId, score
        WHERE gds.util.asNode(nodeId):CD
        WITH gds.util.asNode(nodeId).id AS nodeId, score as score_cont
        RETURN nodeId, score_cont
    }
    RETURN nodeId, score_cont, 0 as score_pref

    UNION 

    CALL {
        CALL gds.eigenvector.stream('searchdomain_preference')
        YIELD nodeId, score
        WHERE gds.util.asNode(nodeId):CD
        WITH gds.util.asNode(nodeId).id AS nodeId, score as score_pref
        RETURN nodeId, score_pref
    }
    RETURN nodeId, 0 as score_cont, score_pref

}

WITH DISTINCT nodeId, count(*) as count, sum(score_cont) as score_cont, sum(score_pref) as score_pref

MATCH (n:CD)
WHERE n.id = nodeId
OPTIONAL MATCH (n)-[:CONTAINS]->(ar:Artist)
OPTIONAL MATCH (n)-[:CONTAINS]->(ab:Album)
OPTIONAL MATCH (n)-[:CONTAINS]->(so:Song)

RETURN nodeId, 
count,
score_cont, score_pref,
collect(DISTINCT ar.artist) as artists,
collect(DISTINCT ab.album) as albums, 
collect(DISTINCT so.song) as songs


ORDER BY score_cont DESC, score_pref DESC
LIMIT 20
"""

dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(30)

,nodeId,count,score_cont,score_pref,artists,albums,songs
0,7923,1,0.301122,0.000000,[signature licks],[jimi hendrix],"[fire (verse);, foxey lady (solo, slow);, fire..."
1,33321,2,0.181730,0.017759,[jimi hendrix],[astro man(alchemy); - studio outtakes 1966-68],"[purple haze 1 (4);, 51st anniversary (5);, la..."
2,30138,2,0.181730,0.017759,[jimi hendrix],[astro man box set],"[can you see me (pre unre);, purple haze (pre ..."
3,20709,1,0.181730,0.000000,[jimi hendrix experience],[the vpro archive - it must be dusty],"[mockingbird (mpeg1 video data);, spoken word,..."
4,2023,2,0.122034,0.034758,[jimi hendrix],[experience hendrix - the best of jimi hendrix],"[bold as love, night bird flying, if 6 was 9, ..."
5,677,2,0.122034,0.034758,[jimi hendrix],[are you experienced],"[third stone from the sun, remember, fire, can..."
6,1936,2,0.122034,0.034758,[jimi hendrix],[experience hendrix],"[bold as love, night bird flying, if 6 was 9, ..."
7,35734,2,0.122034,0.034758,[jimi hendrix],[are you experienced],"[i don't live today, 51st anniversary, stone f..."
8,30933,2,0.122034,0.034758,[jimi hendrix],[the last ever show - love and peace festival ...,"[land of the new rising sun, ezy rider, room f..."
9,136907,2,0.122034,0.034758,[jimi hendrix],[are you experienced],"[love or confusion, remember, highway chile, m..."


***

#### Playground:

* Fuzzy Search did not result in a good performance.
* Combined projection of likes and search input is sensible to number of likes

In [28]:

# ----------------------------------- Nodes matching User Preference -----------------------------------

query_nodes_liked = """
    CALL {
    // Liked Artists, Albums and Songs
    MATCH (u:User)-[:LIKES]->(n) 
    WHERE u.id = 2
        AND (n:Song OR n:Album OR n:Artist) 
    RETURN id(n) AS id, labels(n) AS labels 
    LIMIT 100000
    
    UNION
    
    // CDs linked to liked Artists, Albums and Songs
    MATCH (u:User)-[:LIKES]->(x)-[:APPEARED_ON]->(n:CD) 
    WHERE u.id = 2 
    RETURN id(n) AS id, labels(n) AS labels 
    LIMIT 100000
    }
    RETURN id, labels
"""


# ----------------------------------- Nodes matching Query -----------------------------------

query_nodes_match = """
    
    CALL {
    // Artists, Albums and Songs which match query
    MATCH (n) 
    WHERE 
    (n:Song OR n:Album OR n:Artist) 
    AND 
    (n.artist =~ \'.*jimi hendrix.*\' 
    OR n.album  =~ '.*Are You Experienced.*' 
    OR n.song  =~ '.*purple haze.*')
    RETURN id(n) AS id, labels(n) AS labels 
    LIMIT 1000
    
    UNION
    
    // CDS linked to Artists, Albums and Songs which match query
    MATCH (n)-[:APPEARED_ON]->(c:CD) 
    WHERE 
    (n:Song OR n:Album OR n:Artist) 
    AND 
    (n.artist =~ '.*jimi hendrix.*' 
    OR n.album  =~ '.*Are You Experienced.*' 
    OR n.song  =~ '.*purple haze.*')
    RETURN id(c) AS id, labels(c) AS labels 
    LIMIT 1000
    }
    RETURN id, labels
"""

query_nodes_combined = query_nodes_liked + "UNION " + query_nodes_match


#print(query)

dtf_data = pd.DataFrame([dict(_) for _ in session.run(query_nodes_combined)])
dtf_data.head()

,id,labels
0,56133,[Album]
1,40461,[Album]
2,54581,[Album]
3,39836,[Album]
4,60552,[Album]


In [25]:

# ----------------------------------- Nodes matching User Preference -----------------------------------

query_edges_liked = """
    CALL {
    MATCH (u:User)-[:LIKES]->(n)
    WHERE u.id = 2
    AND (n:CD OR n:Song OR n:Album OR n:Artist) 
    MATCH (n)-[r:APPEARED_ON]->(m:CD) 
    RETURN id(n) AS source, id(m) AS target, type(r) AS type 
    LIMIT 100000
    }
    RETURN source, target, type
"""


# ----------------------------------- Nodes matching Query -----------------------------------

query_edges_match = """

    CALL {
    MATCH (n)-[r:APPEARED_ON]->(c:CD) 
    WHERE 
    (n:Song OR n:Album OR n:Artist) 
    AND 
    (n.artist =~ '.*jimi hendrix.*' 
    OR n.album  =~ '.*Are You Experienced.*' 
    OR n.song  =~ '.*purple haze.*')
    RETURN id(n) AS source, id(c) AS target, type(r) AS type 
    LIMIT 1000
    }
    RETURN source, target, type
    
"""

query_edges_combined = query_edges_liked + " UNION " + query_edges_match


#print(query)

dtf_data = pd.DataFrame([dict(_) for _ in session.run(query_edges_combined)])
dtf_data.head()

,source,target,type
0,56133,902129,APPEARED_ON
1,40461,881559,APPEARED_ON
2,54581,899949,APPEARED_ON
3,39836,905396,APPEARED_ON
4,39836,880789,APPEARED_ON


In [34]:
query_projection_combined = f"""
// CREATE NEW COMBINED PROJECTION
CALL gds.graph.project.cypher(
  "searchdomain_combined",
    " {query_nodes_combined} ", 
    " {query_edges_combined} "
)
YIELD
  graphName, nodeCount AS nodes, relationshipCount AS rels
RETURN graphName, nodes, rels
"""

print(query_projection_combined)

dtf_data = pd.DataFrame([dict(_) for _ in session.run(query_projection_combined)]) 
dtf_data


// CREATE NEW COMBINED PROJECTION
CALL gds.graph.project.cypher(
  "searchdomain_combined",
    " 
    CALL {
    // Liked Artists, Albums and Songs
    MATCH (u:User)-[:LIKES]->(n) 
    WHERE u.id = 2
        AND (n:Song OR n:Album OR n:Artist) 
    RETURN id(n) AS id, labels(n) AS labels 
    LIMIT 100000
    
    UNION
    
    // CDs linked to liked Artists, Albums and Songs
    MATCH (u:User)-[:LIKES]->(x)-[:APPEARED_ON]->(n:CD) 
    WHERE u.id = 2 
    RETURN id(n) AS id, labels(n) AS labels 
    LIMIT 100000
    }
    RETURN id, labels
 	 UNION 
    
    CALL {
    // Artists, Albums and Songs which match query
    MATCH (n) 
    WHERE 
    (n:Song OR n:Album OR n:Artist) 
    AND 
    (n.artist =~ '.*jimi hendrix.*' 
    OR n.album  =~ '.*Are You Experienced.*' 
    OR n.song  =~ '.*purple haze.*')
    RETURN id(n) AS id, labels(n) AS labels 
    LIMIT 1000
    
    UNION
    
    // CDS linked to Artists, Albums and Songs which match query
    MATCH (n)-[:APPEARED_ON]->(c:CD)

,graphName,nodes,rels
0,searchdomain_combined,683,748


In [35]:
# https://neo4j.com/docs/graph-data-science/current/algorithms/eigenvector-centrality/
query = """
CALL gds.eigenvector.mutate('searchdomain_combined',  {
  mutateProperty: 'score_eig'
})
YIELD centralityDistribution, nodePropertiesWritten, ranIterations
RETURN centralityDistribution.min AS minimumScore, centralityDistribution.mean AS meanScore, nodePropertiesWritten
"""
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(30)

,minimumScore,meanScore,nodePropertiesWritten
0,0.001366,0.034714,683


In [38]:
# https://neo4j.com/docs/graph-data-science/current/algorithms/degree-centrality/
query = """
CALL gds.degree.mutate('searchdomain_combined',  {
  mutateProperty: 'score_deg'
})
YIELD centralityDistribution, nodePropertiesWritten
RETURN centralityDistribution.min AS minimumScore, centralityDistribution.mean AS meanScore, nodePropertiesWritten
"""
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(30)

,minimumScore,meanScore,nodePropertiesWritten
0,0.0,1.095171,683


In [39]:
query = """
CALL gds.graph.streamNodeProperties('searchdomain_combined', ['score_eig', "score_deg"])
YIELD nodeId, nodeProperty, propertyValue
WITH nodeId, gds.util.asNode(nodeId).id AS id, nodeProperty, propertyValue

ORDER BY nodeId, nodeProperty
WITH  nodeId, gds.util.asNode(nodeId).id as id, 
collect(nodeProperty) as properties , collect(propertyValue) as values, sum(propertyValue) as sum_values

MATCH (n:CD)
WHERE n.id = id
MATCH (n)-[:CONTAINS]->(ar:Artist)
MATCH (n)-[:CONTAINS]->(ab:Album)
MATCH (n)-[:CONTAINS]->(so:Song)
RETURN id, 
values, sum_values, 
collect(DISTINCT ar.artist) as artists,
collect(DISTINCT ab.album) as albums, 
collect(DISTINCT so.song) as songs
ORDER BY sum_values DESC
LIMIT 100
"""

dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(45)

,id,values,sum_values,artists,albums,songs
0,529,"[74.0, 0.0013661113453287755]",74.001366,[jimi hendrix],[bbc sessions],"[driving south, hear my train a comin, foxy la..."
1,3248,"[53.0, 0.0013661113453287755]",53.001366,[tamba trio],[tamba trio],"[consolacao, batida diferente, imagem, canto d..."
2,35253,"[46.0, 0.0013661113453287755]",46.001366,[erik satie],[satie: 3 gymnopedies & other piano works],"[sonatine bureaucratique, vieux sequins et vie..."
3,452,"[8.0, 0.0013661113453287755]",8.001366,[the jimi hendrix experience],[bbc sessions],"[driving south (alternate take);, wait until t..."
4,20709,"[5.0, 0.0013661113453287755]",5.001366,[jimi hendrix experience],[the vpro archive - it must be dusty],"[mockingbird (mpeg1 video data);, spoken word,..."
5,2835,"[2.0, 0.0013661113453287755]",2.001366,[jimi hendrix (w. lonnie youngblood);],[two great experiencesremastered);],"[she's a fox (1);, groove maker 1 (3);, soul f..."
6,40920,"[2.0, 0.0013661113453287755]",2.001366,[lorenzo perosi],[il natale del redentore],"[il natale, l'annunciazione]"
7,119951,"[2.0, 0.0013661113453287755]",2.001366,[stamic quartet],[janáèek - mozart - schubert],"[erato. adagio. allegro, ante. con moto. alleg..."
8,110321,"[2.0, 0.0013661113453287755]",2.001366,[rajyashree josyer shrikanth],[sangeetha sowrabha],"[brochevarevarura (khamas, adi, mysore vasudev..."
9,50271,"[1.0, 0.0013661113453287755]",1.001366,[ludwig van beethoven],[symphony no.9 choral],"[adagio molto e cantabile, allegro ma non trop..."
